In [4]:
!pip install Pandas pillow tensorflow pyspark pyarrow

In [5]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import os

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split, udf
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.feature import PCA, StandardScaler

In [6]:
print(os.getcwd())

/home/jovyan/work


In [7]:
PATH = os.getcwd()
PATH_Data = PATH+'/fruits-360_dataset/fruits-360/Test/Apple Braeburn'
PATH_Data_2 = PATH+'/fruits-360_dataset/fruits-360/Test_2'
PATH_Result = PATH+'/fruits-360_dataset/fruits-360/Results'
PATH_Result_PCA = PATH+'/fruits-360_dataset/fruits-360/Results_PCA'
PATH_Result_PCA_2 = PATH+'/fruits-360_dataset/fruits-360/Results_PCA_2'
print('PATH:              '+PATH+\
      '\nPATH_Data:       '+PATH_Data+\
      '\nPATH_Data_2:     '+PATH_Data_2+\
      '\nPATH_Result:     '+PATH_Result+\
      '\nPATH_Result_PCA: '+PATH_Result_PCA+\
     '\nPATH_Result_PCA_2:'+PATH_Result_PCA_2)

PATH:              /home/jovyan/work
PATH_Data:       /home/jovyan/work/fruits-360_dataset/fruits-360/Test/Apple Braeburn
PATH_Data_2:     /home/jovyan/work/fruits-360_dataset/fruits-360/Test_2
PATH_Result:     /home/jovyan/work/fruits-360_dataset/fruits-360/Results
PATH_Result_PCA: /home/jovyan/work/fruits-360_dataset/fruits-360/Results_PCA
PATH_Result_PCA_2:/home/jovyan/work/fruits-360_dataset/fruits-360/Results_PCA_2


In [8]:
spark = (SparkSession
             .builder
             .appName('P8')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

In [9]:
sc = spark.sparkContext

In [10]:
spark

In [11]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

In [12]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------------------------------------------------+--------------+
|path                                                                                  |label         |
+--------------------------------------------------------------------------------------+--------------+
|file:/home/jovyan/work/fruits-360_dataset/fruits-360/Test/Apple Braeburn/r_326_100.jpg|Apple Braeburn|
|file:/home/jovyan/work/fruits-360_dataset/fruits-360/Test/Apple Braeburn/r_4_100.jpg  |Apple Braeburn|
|file:/home/jovyan/work/fruits-360_dataset/fruits-360/Test/Apple Braeburn/r_8_100.jpg  |Apple Braeburn|
|file:/home/jovyan/work/fruits-360_dataset/fruits-360/Test/Apple Braeburn/r_324_100.jpg|Apple Braeburn|
|file:/home/jovyan/work/fruits-360_dataset/fruits-360/Test/App

In [13]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

In [14]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

In [15]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [16]:
brodcast_weights = sc.broadcast(new_model.get_weights())

In [17]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

In [18]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

/usr/local/spark/python/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [19]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

In [20]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

In [21]:
print(PATH_Result)

/home/jovyan/work/fruits-360_dataset/fruits-360/Results


In [22]:
#Spark PCA
def array_to_vector(array):
    return Vectors.dense(array)

to_vector_udf = udf(array_to_vector, VectorUDT())

features_vector_df = features_df.withColumn("features_vector", to_vector_udf("features"))

In [23]:
features_vector_df.select('features_vector').show(25)

+--------------------+
|     features_vector|
+--------------------+
|[0.61855274438858...|
|[0.76240664720535...|
|[0.43452119827270...|
|[0.58045774698257...|
|[1.48074543476104...|
|[0.38210460543632...|
|[0.70593112707138...|
|[0.77286916971206...|
|[0.58776515722274...|
|[0.54296451807022...|
|[0.64172405004501...|
|[0.81505554914474...|
|[0.68259793519973...|
|[0.75976765155792...|
|[0.01354158204048...|
|[1.50734794139862...|
|[1.36691784858703...|
|[0.74703872203826...|
|[0.99582183361053...|
|[0.62548857927322...|
|[0.0,0.0,0.0,0.0,...|
|[0.81792730093002...|
|[0.92367011308670...|
|[0.80328118801116...|
|[0.66679257154464...|
+--------------------+
only showing top 25 rows



In [24]:
#StandardScaler
scaler = StandardScaler(inputCol="features_vector", 
                        outputCol="scaled_features", 
                        withMean=True,   # centrer
                        withStd=True)    # réduire

# Étape 2 : Ajuster et transformer les données
scaler_model = scaler.fit(features_vector_df)
scaled_df = scaler_model.transform(features_vector_df)

In [25]:
#dump du standard scaler
SCALER_PATH = PATH+"/models/standard_scaler"

# Sauvegarde du modèle
scaler_model.write().overwrite().save(SCALER_PATH)


In [27]:
#choix de k
pca = PCA(k=100, inputCol="scaled_features", outputCol="pca_features")
model = pca.fit(scaled_df)

explained_variance = model.explainedVariance.toArray()
cumulative_variance = explained_variance.cumsum()
print(f"For k = 100, cumulative explained variance: {cumulative_variance}")

For k = 100, cumulative explained variance: [0.18320215 0.3405928  0.41683611 0.47850163 0.51989908 0.5520948
 0.57704972 0.59967152 0.61744637 0.63361936 0.64833157 0.66252876
 0.67518925 0.68711865 0.69725763 0.70728299 0.71627156 0.72437867
 0.73208208 0.73888094 0.7455104  0.75173493 0.75776125 0.76336112
 0.76880857 0.77395892 0.77897611 0.78362298 0.78821337 0.79273404
 0.79714815 0.80146657 0.80577044 0.80991568 0.81400175 0.81796006
 0.82183949 0.82563577 0.82936965 0.83308896 0.83674977 0.84023667
 0.84369742 0.84712995 0.85049607 0.85379701 0.85699285 0.8601003
 0.86313463 0.86610419 0.86902664 0.87190641 0.87469282 0.87741475
 0.88009453 0.88270277 0.8852752  0.88782427 0.8903258  0.8927906
 0.89520099 0.89755814 0.89987647 0.90212289 0.90431409 0.90647603
 0.90860974 0.91069873 0.91276794 0.91481332 0.91680747 0.91877403
 0.92071812 0.92264778 0.92452033 0.92634541 0.92815727 0.92993809
 0.93168194 0.93338308 0.93505714 0.93670375 0.93831218 0.93984701
 0.94137188 0.9428777

In [41]:
pca = PCA(k=91, inputCol="scaled_features", outputCol="pca_features")  # k = nombre de composantes principales désirées

pca_model = pca.fit(scaled_df)

pca_result = pca_model.transform(scaled_df)


In [42]:
#dump PCA
PCA_PATH = PATH+"/models/pca"

# Sauvegarde du modèle
pca_model.write().overwrite().save(PCA_PATH)

In [43]:
#features_df.write.mode("overwrite").parquet(PATH_Result)

In [44]:
vector_to_array_udf = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

pca_result_clean = pca_result.withColumn("pca_array", vector_to_array_udf("pca_features"))
#pca_result_clean.select("path", "label", "pca_array").show(truncate=False)

In [45]:
pca_result_clean.select("path", "label", "pca_array").write.mode("overwrite").parquet(PATH_Result_PCA)

In [33]:
images_2 = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data_2)

In [34]:
images_2 = images_2.withColumn('label', element_at(split(images_2['path'], '/'),-2))
print(images_2.printSchema())
print(images_2.select('path','label').show(5,False))

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-------------------------------------------------------------------------------------+-----------+
|path                                                                                 |label      |
+-------------------------------------------------------------------------------------+-----------+
|file:/home/jovyan/work/fruits-360_dataset/fruits-360/Test_2/Cauliflower/r_183_100.jpg|Cauliflower|
|file:/home/jovyan/work/fruits-360_dataset/fruits-360/Test_2/Cauliflower/r_173_100.jpg|Cauliflower|
|file:/home/jovyan/work/fruits-360_dataset/fruits-360/Test_2/Cauliflower/r_181_100.jpg|Cauliflower|
|file:/home/jovyan/work/fruits-360_dataset/fruits-360/Test_2/Cauliflower/r_180_100.jpg|Cauliflower|
|file:/home/jovyan/work/fruits-360_dataset/fruits-360/Test_2/Cauliflower/r_185_100.jpg|Cau

In [35]:
features_df_2 = images_2.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

In [36]:
features_vector_df_2 = features_df_2.withColumn("features_vector", to_vector_udf("features"))

In [37]:
features_vector_df_2.select('features_vector').show(25)

+--------------------+
|     features_vector|
+--------------------+
|[0.0,1.1211800575...|
|[0.0,1.4861459732...|
|[0.0,0.5576003789...|
|[0.04578933864831...|
|[0.0,0.3297303318...|
|[0.0,1.9626464843...|
|[0.0,1.0891274213...|
|[0.13888449966907...|
|[0.0,1.5695804357...|
|[0.0,0.3262568414...|
|[8.98555852472782...|
|[0.61725896596908...|
|[0.67813265323638...|
|[0.47783917188644...|
|[0.81906986236572...|
|[0.74139279127120...|
|[0.31645154953002...|
|[1.29626858234405...|
|[1.43449771404266...|
|[0.99171489477157...|
|[1.38247716426849...|
|[1.33058691024780...|
|[0.20126591622829...|
|[0.70482093095779...|
|[1.46605360507965...|
+--------------------+
only showing top 25 rows



In [38]:
#StandardScaler
scaler_2 = StandardScaler(inputCol="features_vector", 
                        outputCol="scaled_features", 
                        withMean=True,   # centrer
                        withStd=True)    # réduire

# Étape 2 : Ajuster et transformer les données
scaler_model_2 = scaler_2.fit(features_vector_df_2)
scaled_df_2 = scaler_model_2.transform(features_vector_df_2)

In [39]:
#dump du standard scaler
SCALER_PATH_2 = PATH+"/models/standard_scaler_2"

# Sauvegarde du modèle
scaler_model_2.write().overwrite().save(SCALER_PATH_2)


In [46]:
#choix de k
pca_2 = PCA(k=200, inputCol="scaled_features", outputCol="pca_features")
model_2 = pca_2.fit(scaled_df_2)

explained_variance_2 = model_2.explainedVariance.toArray()
cumulative_variance_2 = explained_variance_2.cumsum()
print(f"For k = 100, cumulative explained variance: {cumulative_variance_2}")

For k = 100, cumulative explained variance: [0.12697888 0.23494209 0.33225748 0.4060317  0.44806324 0.48091972
 0.51158554 0.53996361 0.56437357 0.58622279 0.6074891  0.62696183
 0.64495339 0.66079216 0.6746374  0.68790229 0.69821825 0.70793667
 0.71688909 0.72450988 0.73192684 0.7390238  0.74603136 0.75260622
 0.75869108 0.76426823 0.76965958 0.77489837 0.77963337 0.78434113
 0.78881081 0.79311066 0.79726155 0.80121187 0.80498318 0.80858893
 0.8120025  0.81524515 0.8183296  0.82128862 0.82421343 0.82695163
 0.82947227 0.83197097 0.83442456 0.83679764 0.83910403 0.8413676
 0.84355061 0.84567078 0.84769648 0.84970912 0.85164226 0.85352756
 0.85537335 0.85717531 0.85897215 0.86072919 0.86242563 0.86409069
 0.86572209 0.8673067  0.86887342 0.87037332 0.87185719 0.87330099
 0.87472519 0.87611841 0.87746871 0.87881118 0.88010341 0.88135644
 0.88260121 0.88379351 0.88497918 0.88614781 0.88729159 0.88841969
 0.88952509 0.89062326 0.89168169 0.89272521 0.89375892 0.89477453
 0.89577446 0.89677

In [47]:
pca_2 = PCA(k=75, inputCol="scaled_features", outputCol="pca_features")  # k = nombre de composantes principales désirées

pca_model_2 = pca_2.fit(scaled_df_2)

pca_result_2 = pca_model_2.transform(scaled_df_2)


In [48]:
#dump PCA
PCA_PATH_2 = PATH+"/models/pca_2"

# Sauvegarde du modèle
pca_model_2.write().overwrite().save(PCA_PATH_2)

In [49]:
vector_to_array_udf = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

pca_result_clean_2 = pca_result_2.withColumn("pca_array", vector_to_array_udf("pca_features"))
#pca_result_clean_2.select("path", "label", "pca_array").show(truncate=False)

In [50]:
pca_result_clean_2.select("path", "label", "pca_array").write.mode("overwrite").parquet(PATH_Result_PCA_2)

In [51]:
df_2 = pd.read_parquet(PATH_Result_PCA_2, engine='pyarrow')

In [52]:
df_2.head()

,path,label,pca_array
0,file:/home/jovyan/work/fruits-360_dataset/frui...,Cauliflower,"[23.266771, -2.0451193, -0.73159224, -4.394127..."
1,file:/home/jovyan/work/fruits-360_dataset/frui...,Cauliflower,"[22.517595, -1.8714422, -0.5529274, -4.7130713..."
2,file:/home/jovyan/work/fruits-360_dataset/frui...,Cauliflower,"[24.567892, -1.6081315, -2.0830262, -5.4299235..."
3,file:/home/jovyan/work/fruits-360_dataset/frui...,Cauliflower,"[18.89023, -4.037467, -2.9732332, -3.7888887, ..."
4,file:/home/jovyan/work/fruits-360_dataset/frui...,Cauliflower,"[17.179165, -7.1081433, 1.26955, -1.2055696, -..."


In [53]:
df_2.loc[0,'pca_array'].shape

(75,)

In [54]:
df = pd.read_parquet(PATH_Result_PCA, engine='pyarrow')

In [55]:
df.head()

,path,label,pca_array
0,file:/home/jovyan/work/fruits-360_dataset/frui...,Apple Braeburn,"[-14.683198, 4.102032, 15.725445, -7.6482406, ..."
1,file:/home/jovyan/work/fruits-360_dataset/frui...,Apple Braeburn,"[-7.78049, -7.3234935, 13.995015, -11.8683605,..."
2,file:/home/jovyan/work/fruits-360_dataset/frui...,Apple Braeburn,"[-21.829105, 9.647164, -19.66954, -17.515055, ..."
3,file:/home/jovyan/work/fruits-360_dataset/frui...,Apple Braeburn,"[14.975967, 1.138271, -1.437785, 1.6589977, 3...."
4,file:/home/jovyan/work/fruits-360_dataset/frui...,Apple Braeburn,"[-12.944075, 12.881019, -4.830613, 16.8954, 8...."


In [56]:
df.loc[0,'pca_array'].shape

(91,)

In [33]:
df_result = pd.read_parquet(PATH_Result, engine='pyarrow')

In [34]:
df_result.head()

,path,label,features
0,file:/home/jovyan/work/fruits-360_dataset/frui...,Apple Braeburn,"[0.6685475, 0.113827415, 0.0, 0.0, 0.0, 1.0088..."
1,file:/home/jovyan/work/fruits-360_dataset/frui...,Apple Braeburn,"[0.63903767, 0.036415994, 0.0, 0.0, 0.0, 0.865..."
2,file:/home/jovyan/work/fruits-360_dataset/frui...,Apple Braeburn,"[1.049216, 0.15010181, 0.0, 0.0, 0.0, 0.784024..."
3,file:/home/jovyan/work/fruits-360_dataset/frui...,Apple Braeburn,"[0.58045775, 0.21252142, 0.0, 0.0, 0.04099558,..."
4,file:/home/jovyan/work/fruits-360_dataset/frui...,Apple Braeburn,"[1.5858144, 0.174534, 0.0, 0.0, 0.37722304, 0...."


In [35]:
df_result.loc[0,'features'].shape

(1280,)